In [7]:
import pandas as pd
import glob
import numpy as np
from sqlalchemy import create_engine
import warnings
from geopy.geocoders import Nominatim

# Ignore Warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
ht_path = "High Tension/"
lt_path = "Low Tension/"
ht_files = glob.glob(ht_path + "*.csv")
lt_files = glob.glob(lt_path + "*.csv")
ht_files.sort()
lt_files.sort()

In [3]:
ht = []
lt = []
for i in range(len(ht_files)):
  ht.append(pd.read_csv(ht_files[i]))
  lt.append(pd.read_csv(lt_files[i]))

In [4]:
for i in range(len(ht)):
    ht[i]['Divisions'] = ht[i]['Divisions'] + ", Bengaluru"
    lt[i]['Divisions'] = lt[i]['Divisions'] + ", Bengaluru"

In [6]:
for i in range(len(ht)):
    ht[i].dropna(inplace=True)
    lt[i].dropna(inplace=True)